# Privacy-Preserving Techniques for LLM Code Completion: Andrei Oprescu

This jupyter notebook is dedicated to completing the task for the Privacy-Preserving Techniques for LLM Code Completion JetBrains task. Below you can find the submission.

First, lets install the dependencies and import the required libraries

In [ ]:
%pip install transformers torch
%pip install pandas
%pip install datasets
%pip install numpy
%pip install rouge-score
%pip install matplotlib
%pip install accelerate
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
from datasets import load_dataset
import re
import pandas
import numpy as np
from rouge_score import rouge_scorer
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

### Data Fetching and Preprocessing

Next, we will get the dataset from HuggingFace.

In [ ]:
dataset = load_dataset("openai_humaneval", split="test")
dataset.set_format("pandas")
print("Length: ", len(dataset))

Get the first 20 examples for the code completion and print one out.

In [ ]:
first20 = dataset[:20][["task_id", "prompt", "canonical_solution"]]
print(first20.iloc[1])

For the low obfuscation version, I decided to remove the variable and method data types. This should not affect the code completion model's ability to understand the task too much, since it can still make out what the variable and method are supposed to be used for from the method name, comments, and variable name.

In [ ]:
def remove_types(match):
    # Clean the signature string found by the main regex
    signature = match.group(0)
    # Remove param types by looking for the colon
    signature = re.sub(r":\s*[^,)=]+", "", signature)
    # Remove return type by looking for the ->
    signature = re.sub(r"->\s*[^:]+", "", signature)
    return signature

def low_obfuscation(df: pandas.DataFrame) -> pandas.DataFrame:
    # Define the method signature regex expression
    signature_pattern = r"def\s+\w+\s*\(.*?\)\s*->\s*[^:]+:|def\s+\w+\s*\(.*?\):"
    # Make a copy to not edit the original DataFrame
    df_copy = df.copy()
    
    # Make helper to handle regex for each prompt
    def process_prompt(text):
        return re.sub(signature_pattern, remove_types, text)
                      
    # Create new column "edited_prompt" with apply and helper
    df_copy['edited_prompt'] = df_copy["prompt"].apply(process_prompt)
    return df_copy


Now let's create the low-obfuscated prompts and save them.

In [ ]:
# Get the low-obfuscated prompts and print one
first20_low_obfuscation = low_obfuscation(first20)
print(first20_low_obfuscation.iloc[1]["edited_prompt"])

For the high-obfuscation, I decided to just remove the single-line and multi-line comments from the method. This should make it harder for the code completion model to understand how to autocomplete the code, since pretty much the only piece of information it has about its task is the method signature.

In [ ]:
def high_obfuscation(df: pandas.DataFrame) -> pandas.DataFrame:
    # Define the multi-line and single line regex expressions
    doc_pattern = r"\"\"\".*?\"\"\"|'''.*?'''"
    inline_pattern = r"#.*?$"
    # Make a copy of the DataFrame to not override the prompts from the original DataFrame
    df_copy = df.copy()
    
    # Make helper for removing comments
    def clean_comments(text):
        text = re.sub(doc_pattern, "", text, flags=re.DOTALL)
        text = re.sub(inline_pattern, "", text, flags=re.MULTILINE)
        return text

    # Create new column "edited_prompt" with apply and helper
    df_copy['edited_prompt'] = df_copy["prompt"].apply(clean_comments)

    return df_copy


Create the high-obfuscated prompts and save them.

In [ ]:
first20_high_obfuscation = high_obfuscation(first20)
print(first20_high_obfuscation.iloc[1]["edited_prompt"])

Let's also make the 'edited_prompt' column for the original prompt df for consistency.

In [ ]:
first20['edited_prompt'] = first20['prompt']
print(first20.iloc[1]["edited_prompt"])

### Code Autocompletion

Next, we will get a model from HuggingFace to autocomplete the method functionality. I chose a model with less parameters, since the obfuscation methods should have a higher effect on its performance since it's harder for it to analyse the task, allowing us to see a bigger effect of these obfuscation methods. Additionally, this improved the download speed of the model as well as inference times. The model parameter count can be changed by you by changing the "3B" part of the model_name to any of the following values:  0.5B, 1.5B, 3B, 7B, 14B, 32B

In [ ]:
# Choose a low-parameter model (1.5B parameters)
model_name = "Qwen/Qwen2.5-Coder-3B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device: ", device)

# Load tokenizer and model and use the GPU as the device
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map=device
)



Next, I created a method for completing the code of the 20 prompts in the DataFrame. Since I gave the model no other information, I defined a list of stop strings. The model will stop writing more code once it writes these keywords. Namely, it will stop if it tries to create another method, class or run the code it created in an "if \_\_name\_\_ == '\_\_init\_\_':" statement.



In [ ]:
stop_strings = ["\ndef", "\nclass", "\nif __name__"]

def complete_code(df: pandas.DataFrame) -> pandas.DataFrame:
  # Store the completions
  completions = []

  for _, row in df.iterrows():
    # Tokenize the prompt
    inputs = tokenizer(row['edited_prompt'], return_tensors="pt").to(model.device)
    prompt_length = inputs.input_ids.shape[1]
    # Generate the output of the model for the prompt
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        tokenizer=tokenizer,
        stop_strings=stop_strings,
        temperature=0.1, # Lower temperature for more deterministic and accurate code
        do_sample=True
    )

    # Decode and store the result
    completion_tokens = outputs[0][prompt_length:]
    completion = tokenizer.decode(completion_tokens, skip_special_tokens=True)
    completions.append((completion, row['canonical_solution']))

  return completions

Next, let's get the code completions of the original, low-obfuscated and high-obfuscated prompts.

In [ ]:
first20_completions = complete_code(first20)
first20_low_obfuscation_completions = complete_code(first20_low_obfuscation)
first20_high_obfuscation_completions = complete_code(first20_high_obfuscation)

### Evaluation

I first created the methods for normalized Levenshtein for privacy score and ROUGE score for utility score. I implemented the normalized Levenshtein algorithm myself. For the ROUGE scores, I decided to use a library which calculates them itself.

For the ROUGE score, I decided to look at the longest common subsequence (ROUGE-L). I retrieved the precisions, recalls and F-scores of the completions.

In [ ]:
def normalized_levenshtein(str1: str, str2: str) -> float:
  # Get lengths
  l1 = len(str1)
  l2 = len(str2)

  # Create the memoization 2-D array
  mem = np.zeros((l1 + 1, l2 + 1))

  # Fill in the base values for both words
  for i in range(l1 + 1):
    mem[i][0] = i

  for j in range(l2 + 1):
    mem[0][j] = j

  # Fill in the matrix
  for i in range(1, l1 + 1):
    for j in range(1, l2 + 1):
      if str1[i - 1] == str2[j - 1]:
        # If the characters match
        mem[i][j] = mem[i - 1][j - 1]
      else:
        # If the characters don't match
        mem[i][j] = 1 + max(mem[i - 1][j], max(mem[i][j - 1], mem[i - 1][j - 1]))

  # Get the distance between the two words
  distance = mem[l1][l2]

  # Return the normalized distance
  return distance / max(l1, l2)


def perform_levenshtein(df: pandas.DataFrame) -> int:
  """
    Perform normalized Levenshtein distance on completions-solution tuple pairs.
    Return the average Levenshtein distance.
  """
  total = 0
  distances = []
  for _, row in df.iterrows():
    distance = normalized_levenshtein(row['prompt'], row['edited_prompt'])
    distances.append(distance)
    total += distance

  return distances

def get_rouge_score(completions):
  """
    Get the ROUGE score of the completion-solution tuple pairs.
    Return precisions, recalls and f-measures.
  """
  precisions = []
  recalls = []
  fmeasures = []

  n = len(completions)

  for completion, solution in completions:
    # Create the ROUGE scorer and get the score for the completion
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(completion, solution)

    # Store the results
    for key, value in scores.items():
        print(key, value)
        precisions.append(value.precision)
        recalls.append(value.recall)
        fmeasures.append(value.fmeasure)

  # Return the precision, recall and F-measure scores
  return precisions, recalls, fmeasures


Next, let's get the scores for the original, low-obfuscated and high-obfuscated prompts and group them by score type. I averaged the measures so that the plot can show the big picture of the performance of the models.

In [ ]:
# Get the scores
f20_precisions, f20_recalls, f20_fmeasures = get_rouge_score(first20_completions)
low_obfuscation_precisions, low_obfuscation_recalls, low_obfuscation_fmeasures = get_rouge_score(first20_low_obfuscation_completions)
high_obfuscation_precisions, high_obfuscation_recalls, high_obfuscation_fmeasures = get_rouge_score(first20_high_obfuscation_completions)

# Group by score type
precisions = [np.average(f20_precisions), np.average(low_obfuscation_precisions), np.average(high_obfuscation_precisions)]
recalls = [np.average(f20_recalls), np.average(low_obfuscation_recalls), np.average(high_obfuscation_recalls)]
fmeasures = [np.average(f20_fmeasures), np.average(low_obfuscation_fmeasures), np.average(high_obfuscation_fmeasures)]


Finally, I created a method for plotting these scores on a bar graph to showcase the difference in code completion accuracy.

In [ ]:
def plot_scores(precisions, recalls, fmeasures):
  # Define the plot categories
  categories = ["No obfuscation", "Low Obfuscation", "High Obfuscation"]
  x = np.arange(len(categories))
  # Set the width of the bars
  width = 0.2

  fig, ax = plt.subplots()

  # Plot the bars with manual offsets
  ax.bar(x - width, precisions, width, label='Precision', color='yellow', edgecolor='grey')
  ax.bar(x, recalls, width, label='Recall', color='blue', edgecolor='grey')
  ax.bar(x + width, fmeasures, width, label='F-Score', color='green', edgecolor='grey')

  ax.set_ylabel('Scores')
  ax.set_title('Grouped Bar Chart Example')
  ax.set_xticks(x)
  ax.set_xticklabels(categories)
  ax.legend()

  # Plot the bar graph
  plt.show()


Next, let's print out the ROUGE scores in a table and plot the graph. I also displayed the standard deviation of the scores so that we can see how much the utility scores vary with the obfuscations.

In [ ]:
# Plot the scores
plot_scores(precisions, recalls, fmeasures)

# Define the data for the table
data = {
    'Original': [np.average(f20_precisions), np.average(f20_recalls), np.average(f20_fmeasures)],
    'Low Obfuscation': [np.average(low_obfuscation_precisions), np.average(low_obfuscation_recalls), np.average(low_obfuscation_fmeasures)],
    'High Obfuscation': [np.average(high_obfuscation_precisions), np.average(high_obfuscation_recalls), np.average(high_obfuscation_fmeasures)]
}

# Define the row labels
row_titles = ['Precision', 'Recall', 'F-Score']

# Create the DataFrame
df = pandas.DataFrame(data, index=row_titles)

print(df)
print(f"Original prompt F-measure standard deviation: {np.std(f20_fmeasures)}")
print(f"Low obfuscation prompt F-measure standard deviation: {np.std(low_obfuscation_fmeasures)}")
print(f"High obfuscation prompt F-measure standard deviation: {np.std(high_obfuscation_fmeasures)}")

As we can see from the plot and the table, the less obfuscated code is autocompleted by the code completion model with a high accuracy, achieving an F-Score similar to the original prompt. However, the F-score can also be higher for the low obfuscated prompt than the original prompt. This most likely happens because of the uncertainty of the model; since the obfuscation does not greatly sabotage the model's ability to understand the task to a great extent, it might end up producing better results on occasional cases, but should not end up being overall better if we used more prompts to assess the effect of obfuscation on utility.

However, when we obfuscate the code more (by removing the comments), the task given to the model is less obvious, which causes the F-score to drop substantially. This is expected, as the model has less information to work with.

Therefore, we see the utility of the code to drop as we increase the obfuscation.

Now let's get the levenshtein distances for each version of the prompts.

In [ ]:
levenshtein_first20 = perform_levenshtein(first20)
levenshtein_first20_low_obfuscation = perform_levenshtein(first20_low_obfuscation)
levenshtein_first20_high_obfuscation = perform_levenshtein(first20_high_obfuscation)

In [ ]:
print("Original prompt Levenshtein score (average): ", np.average(levenshtein_first20))
print("Low-obfuscation prompt Levenshtein score (average): ", np.average(levenshtein_first20_low_obfuscation))
print("High-obfuscation prompt Levenshtein score (average): ", np.average(levenshtein_first20_high_obfuscation))

From the normalized Levenshtein distances, we can see that a low-obfuscation of the code affects the privacy very little. However, when we apply a high-obfuscation method, the normalized Levenshtein distances get bigger by a considerable amount.


Next, let's plot the Utility score on the Privacy score in a scatterplot.

In [ ]:
# Create the plot
plt.figure(figsize=(10, 6))

# Plot Original Data
plt.scatter(levenshtein_first20, f20_fmeasures, color='green', label='Original', alpha=0.6)

# Plot Low Obfuscation
plt.scatter(levenshtein_first20_low_obfuscation, low_obfuscation_fmeasures, color='yellow', label='Low Obfuscation',
            alpha=0.8, edgecolors='black', linewidths=0.5)

# Plot High Obfuscation
plt.scatter(levenshtein_first20_high_obfuscation, high_obfuscation_fmeasures, color='red', label='High Obfuscation', alpha=0.8)

all_x = np.concatenate([levenshtein_first20, levenshtein_first20_low_obfuscation, levenshtein_first20_high_obfuscation])
all_y = np.concatenate([f20_fmeasures, low_obfuscation_fmeasures, high_obfuscation_fmeasures])

# Calculate the line of best fit
coefficients = np.polyfit(all_x, all_y, 1)
polynomial = np.poly1d(coefficients)

# Make a range of x values for the line of best fit
x_line = np.linspace(all_x.min(), all_x.max(), 100)
y_line = polynomial(x_line)

plt.plot(x_line, y_line, color='purple', linestyle='--', linewidth=2, label=f'Trend (Best Fit)')

# Adding labels and styling
plt.xlabel('Privacy Score (Normalized Levenshtein Distance)')
plt.ylabel('Utility Score (ROUGE Score)')
plt.title('Privacy-Utility Trade-off in Code Completion')

plt.legend()
plt.grid(True, linestyle='--', alpha=0.3)

From the plot, we can see that as the Privacy score increases, the Utility score decreases, as shown by the trend line. As mentioned before, this makes sense, as the more information we remove from the prompt (the higher the privacy), the less useful answers we receive (the lower the utility). Additionally, the red points (highly obfuscated prompts) tend to be lower on the utility score and higher on the normalized Levenshtein score compared to the yellow and green points (less obfuscated and original prompts)